# Introduction

This tutorial demonstrates how to achieve phase and orientation mapping via scanning electron diffraction using both pattern and vector matching.

The data was acquired from a GaAs nanowire displaying polymorphism between zinc blende and wurtzite structures.

This functionaility has been checked to run in pyxem-0.9.0 (July 2019). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

1. <a href='#loa'> Load & Inspect Data</a>
2. <a href='#pre'> Pre-processing</a>
3. <a href='#tem'> Template matching</a>
    1. <a href='#tema'> [Build Template Library]</a>
    2. <a href='#temb'>[Indexing]</a>
4. <a href='#vec'> Vector Matching</a>
    1. <a href='#veca'> [Build Vector Library]</a>
    2. <a href='#vecb'>[Indexing Vectors]</a>

Import pyXem and other required libraries

In [ ]:
%matplotlib qt
import math
import numpy as np
import diffpy

import pyxem as pxm

Download and the data for this demo from here and put in directory with notebooks:

https://drive.google.com/drive/folders/1nkqDIu8g_kOQOuRqx5yDigtjopgv_Isj?usp=sharing

# <a id='loa'></a> 1. Loading and Inspection

Load the demo data

In [ ]:
dp = pxm.load_hspy('./NW_GaAs_ZB_WZ_pyxem_sample.hdf5',
                   assign_to='electron_diffraction2d')
dp

Crop a subset of the data to be analyzed

In [ ]:
dp = dp.inav[90:110, 30:75]
dp

Set data type, scale intensity range and set calibration

In [ ]:
dp.data = dp.data.astype('float64')
dp.data *= 1 / dp.data.max()

Inspect metadata

In [ ]:
dp.metadata

Plot an interactive virtual image to inspect data

In [ ]:
roi = pxm.roi.CircleROI(cx=72, cy=72, r_inner=0, r=2)
dp.plot_interactive_virtual_image(roi=roi, cmap='viridis')

# <a id='pre'></a> 2. Pre-processing

Apply affine transformation to correct for off axis camera geometry

In [ ]:
scale_x = 0.995
scale_y = 1.031
offset_x = 0.631
offset_y = -0.351
dp.apply_affine_transformation(np.array([[scale_x, 0, offset_x],
                                         [0, scale_y, offset_y],
                                         [0, 0, 1]]))

Perform difference of gaussian background subtraction with various parameters on one selected diffraction pattern and plot to identify good parameters

In [ ]:
dp_test_area = dp.inav[0, 0]

gauss_stddev_maxs = np.arange(2, 12, 0.2) # min, max, step
gauss_stddev_mins = np.arange(1, 4, 0.2) # min, max, step
gauss_processed = np.empty((
len(gauss_stddev_maxs),
len(gauss_stddev_mins),
*dp.axes_manager.signal_shape))

for i, gauss_stddev_max in enumerate(gauss_stddev_maxs):
    for j, gauss_stddev_min in enumerate(gauss_stddev_mins):
        gauss_processed[i, j] = dp_test_area.remove_background('gaussian_difference',
                                                               sigma_min=gauss_stddev_min,
                                                               sigma_max=gauss_stddev_max,
                                                               show_progressbar=False)
dp_gaussian = pxm.ElectronDiffraction2D(gauss_processed)
dp_gaussian.metadata.General.title = 'Gaussian preprocessed'
dp_gaussian.axes_manager.navigation_axes[0].name = r'$\sigma_{\mathrm{min}}$'
dp_gaussian.axes_manager.navigation_axes[0].offset = gauss_stddev_mins[0]
dp_gaussian.axes_manager.navigation_axes[0].scale = gauss_stddev_mins[1] - gauss_stddev_mins[0]
dp_gaussian.axes_manager.navigation_axes[0].units = ''
dp_gaussian.axes_manager.navigation_axes[1].name = r'$\sigma_{\mathrm{max}}$'
dp_gaussian.axes_manager.navigation_axes[1].offset = gauss_stddev_maxs[0]
dp_gaussian.axes_manager.navigation_axes[1].scale = gauss_stddev_maxs[1] - gauss_stddev_maxs[0]
dp_gaussian.axes_manager.navigation_axes[1].units = ''

dp_gaussian.plot(cmap='viridis')

Remove background using difference of gaussians method with parameters identified above

In [ ]:
dp = dp.remove_background('gaussian_difference',
                          sigma_min=2, sigma_max=8)
dp.data -= dp.data.min()
dp.data *= 1 / dp.data.max()

Set diffraction calibration and scan calibration

In [ ]:
diffraction_calibration = 0.032
dp.set_diffraction_calibration(diffraction_calibration)
dp.set_scan_calibration(10)

# <a id='tem'></a> 3. Pattern Matching

Pattern matching generates a database of simulated diffraction patterns and then compares all simulated patterns against each experimental pattern to find the best match

Import generators required for simulation and indexation

In [ ]:
from diffsims.generators.structure_library_generator import StructureLibraryGenerator
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator

from pyxem.generators.indexation_generator import IndexationGenerator

## 3.1. Define Library of Structures & Orientations

Define the crystal phases to be included in the simulated library

In [ ]:
structure_zb = diffpy.structure.loadStructure('./GaAs_mp-2534_conventional_standard.cif')
structure_wz = diffpy.structure.loadStructure('./GaAs_mp-8883_conventional_standard.cif')

phase_descriptions = [('ZB', structure_zb, 'cubic'),
                      ('WZ', structure_wz, 'hexagonal')]
phase_names = [phase[0] for phase in phase_descriptions]

Initialize a structure library generator for the specified phases

In [ ]:
struc_lib_gen = StructureLibraryGenerator(phase_descriptions)

Construct a StructureLibrary defining crystal structures and orientations for which diffraction will be simulated 

In [ ]:
inplane_rotations = [[0], [0]]  # The library only needs the base in-plane rotation. The other ones are generated
rotation_list_resolution = 1

struc_lib = struc_lib_gen.get_orientations_from_stereographic_triangle(
                            inplane_rotations, rotation_list_resolution)

## <a id='temb'></a> 3.2. Simulate Diffraction for all Structures & Orientations

Define a diffsims DiffractionGenerator with diffraction simulation parameters

In [ ]:
diff_gen = DiffractionGenerator(accelerating_voltage = 200,
                                max_excitation_error=1/10)

Initialize a diffsims DiffractionLibraryGenerator

In [ ]:
lib_gen = DiffractionLibraryGenerator(diff_gen)

Calulate library of diffraction patterns for all phases and unique orientations

In [ ]:
target_pattern_dimension_pixels = dp.axes_manager.signal_shape[0]
half_size = target_pattern_dimension_pixels // 2
reciprocal_radius = diffraction_calibration*(half_size - 1)

diff_lib = lib_gen.get_diffraction_library(struc_lib,
                                           calibration=diffraction_calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=(half_size, half_size),
                                           with_direct_beam=False)

Optionally, save the library for later use.

In [ ]:
diff_lib.pickle_library('./GaAs_cubic_hex_1deg.pickle')

If saved, the library can be loaded as follows

In [ ]:
from diffsims.libraries.diffraction_library import load_DiffractionLibrary

diff_lib = load_DiffractionLibrary('./GaAs_cubic_hex_1deg.pickle', safety=True)

## <a id='temb'></a> 3.3. Pattern Matching Indexation

Initialize `IndexationGenerator` with the experimental data and diffraction library and perform correlation, returning the `n_largest` matches with highest correlation.

In [ ]:
indexer = IndexationGenerator(dp, diff_lib)
indexation_results = indexer.correlate(n_largest=4)

Get crystallographic map from indexation results

In [ ]:
crystal_map = indexation_results.get_crystallographic_map()

Plot the best matching phase as a function of position and corresponding reliablity

In [ ]:
crystal_map.get_phase_map().plot()
crystal_map.get_metric_map('phase_reliability').plot()

Plot the best matching crystal orientation as a function of position and corresponding reliability

In [ ]:
crystal_map.get_orientation_map().plot()
crystal_map.get_metric_map('orientation_reliability').plot()

Plot the best matching results on the signal to inspect

In [ ]:
indexation_results.inav[0:20, 10:30].plot_best_matching_results_on_signal(
    dp.inav[0:20, 10:30], diff_lib, diff_gen, reciprocal_radius)

Optionally, save the crystallographic mapping results for analysis using the open-source matlab package MTEX (https://mtex-toolbox.github.io/)

In [ ]:
crystal_map.save_mtex_map('pattern_match_results.csv')

# <a id='vec'></a> 4. Vector Matching

Vector matching generates a database of vector pairs (magnitues and inter-vector angles) and then compares all theoretical values against each measured diffraction vector pair to find the best match

Import generators required for simulation and indexation

In [ ]:
from diffsims.generators.library_generator import VectorLibraryGenerator
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.libraries.vector_library import load_VectorLibrary

from pyxem.generators.indexation_generator import VectorIndexationGenerator

from pyxem.generators.subpixelrefinement_generator import SubpixelrefinementGenerator
from pyxem.signals.diffraction_vectors import DiffractionVectors

## <a id='veca'></a> 4.1. Define Library of Structures

Define crystal structure for which to determine theoretical vector pairs

In [ ]:
structure_zb = diffpy.structure.loadStructure('./GaAs_mp-2534_conventional_standard.cif')
structure_wz = diffpy.structure.loadStructure('./GaAs_mp-8883_conventional_standard.cif')

structure_library = StructureLibrary(['ZB', 'WZ'],
                                     [structure_zb, structure_wz],
                                     [[], []])

Initialize VectorLibraryGenerator with structures to be considered

In [ ]:
vlib_gen = VectorLibraryGenerator(structure_library)

Determine VectorLibrary with all vectors within given reciprocal radius

In [ ]:
reciprocal_radius = diffraction_calibration*(half_size - 1)

vec_lib = vlib_gen.get_vector_library(reciprocal_radius)

Optionally, save the library for later use

In [ ]:
vec_lib.pickle_library('./GaAs_cubic_hex_vectors.pickle')

In [ ]:
vec_lib = load_VectorLibrary('./GaAs_cubic_hex_vectors.pickle',
                             safety=True)

## 4.2. Find Diffraction Peaks

Tune peak finding parameters interactively

In [ ]:
dp.find_peaks_interactive(imshow_kwargs={'cmap': 'viridis', 'vmax': 0.8})

Perform peak finding on the data with parameters from above

In [ ]:
peaks = dp.find_peaks(method='difference_of_gaussians',
                      min_sigma=0.005,
                      max_sigma=5.0,
                      sigma_ratio=2.0,
                      threshold=0.06,
                      overlap=0.8)

Remove any peeks that are too long and the direct beam

In [ ]:
def filter_peaks(peaks):
    peaks = peaks[0]
    # Only keep vectors within max length and remove centre (closer than 5px to image centre)
    return peaks[(np.linalg.norm(peaks, axis=1) < reciprocal_radius) &
                 (np.any(np.abs(peaks) > 5 * diffraction_calibration, axis=1))]

peaks.map(filter_peaks)
# Map changes the signal type. Reset
peaks = DiffractionVectors(peaks.data)
peaks.axes_manager.set_signal_dimension(0)

Refine the peak positions to sub-pixel precision using the center of mass method of a `SubpixelrefinementGenerator`

In [ ]:
subpixel_refinement = SubpixelrefinementGenerator(dp, peaks)
peaks = DiffractionVectors(subpixel_refinement.center_of_mass_method(square_size=8))
peaks.axes_manager.set_signal_dimension(0)

`peaks` now contain the 2D positions of the diffraction spots on the detector. The vector matching method works in 3D coordinates, which are found by projecting the detector positions back onto the Ewald sphere.

In [ ]:
peaks.calculate_cartesian_coordinates(accelerating_voltage=200,
                                      camera_length=0.2)

## <a id='vecb'></a> 4.3. Vector Matching Indexation

Initialize `VectorIndexationGenerator` with the experimental data and vector library and perform indexation using `n_peaks_to_index` and returning the `n_best` indexation results.

In [ ]:
indexation_generator = VectorIndexationGenerator(peaks, vec_lib)

In [ ]:
indexation_results = indexation_generator.index_vectors(mag_tol=3*diffraction_calibration,
                                                angle_tol=4,  # degree
                                                index_error_tol=0.2,
                                                n_peaks_to_index=7,
                                                n_best=2,
                                                show_progressbar=True)

Get crystallographic map from indexation results

In [ ]:
crystal_map = indexation_results.get_crystallographic_map()

Plot the best matching phase as a function of position and corresponding reliability

In [ ]:
crystal_map.get_phase_map().plot()
crystal_map.get_metric_map('phase_reliability').plot()

Plot the best matching crystal orientation as a function of position and corresponding reliability

In [ ]:
crystal_map.get_orientation_map().plot()
crystal_map.get_metric_map('orientation_reliability').plot()

Plot the best matching results on the signal to inspect

In [ ]:
indexation_results.inav[0:20, 10:30].plot_best_matching_results_on_signal(
    dp.inav[0:20, 10:30], diff_lib, diff_gen, reciprocal_radius)

Optionally, save the crystallographic mapping results for analysis using the open-source matlab package MTEX (https://mtex-toolbox.github.io/)

In [ ]:
crystal_map.save_mtex_map('vector_match_results.csv')